In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
import plotly
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
rank = pd.read_csv('../data/rank_file.csv')

In [3]:
alldata = rank
rank.head()

,Unnamed: 0,City,State,Population,Crime_rank,Hospital_rank,Early_education_rank,University_education_rank,Human_related_rank,Air,...,Economy_rank,Bar_Rank,Restaurant_Rank,Museums_Rank,Libraries_Rank,Park_Rank,TopRes_Rank,Tertiary_Rank,Latitude,Longitude
0,0,Atlanta,GA,410606.0,33.0,22.0,29.0,16.5,27.0,36.0,...,34.0,21.0,21.0,10.0,14.0,28.0,6.0,16.0,33.749099,-84.390185
1,1,Austin,TX,797215.0,3.0,2.5,22.5,14.0,5.0,7.0,...,4.0,15.0,12.0,19.0,19.5,4.0,8.5,8.0,30.271129,-97.743700
2,2,Baltimore,MD,612701.0,40.0,36.0,20.0,20.0,36.0,35.0,...,25.0,13.0,19.0,9.0,16.0,24.5,21.5,17.0,39.290882,-76.610759
3,3,Birmingham,AL,212157.0,45.0,38.0,35.0,36.0,48.0,43.0,...,47.0,34.0,34.0,34.0,34.0,34.0,22.5,42.0,33.520682,-86.802433
4,4,Boston,MA,615462.0,18.0,30.0,4.0,9.0,10.0,22.0,...,7.0,24.0,14.0,8.0,6.0,24.5,6.0,11.0,42.360482,-71.059568


In [5]:
list(rank.columns.values)

['Unnamed: 0',
 'City',
 'State',
 'Population',
 'Crime_rank',
 'Hospital_rank',
 'Early_education_rank',
 'University_education_rank',
 'Human_related_rank',
 'Air',
 'Water',
 'Toxics',
 'Hazardous',
 'Green_score',
 'Natural_total_rank',
 'Rank_unemployment',
 'Rank_sale_rate',
 'Rank_Income',
 'Rank_Tuition',
 'Economy_rank',
 'Bar_Rank',
 'Restaurant_Rank',
 'Museums_Rank',
 'Libraries_Rank',
 'Park_Rank',
 'TopRes_Rank',
 'Tertiary_Rank',
 'Latitude',
 'Longitude']

In [16]:
available = list(rank.columns.values)
for i in ['Unnamed: 0','City','State','Population','Natural_total_rank', 'Human_related_rank', 'Economy_rank', 'Tertiary_Rank', 'Latitude', 'Longitude']:
    available.remove(i)

['Crime_rank',
 'Hospital_rank',
 'Early_education_rank',
 'University_education_rank',
 'Air',
 'Water',
 'Toxics',
 'Hazardous',
 'Green_score',
 'Rank_unemployment',
 'Rank_sale_rate',
 'Rank_Income',
 'Rank_Tuition',
 'Bar_Rank',
 'Restaurant_Rank',
 'Museums_Rank',
 'Libraries_Rank',
 'Park_Rank',
 'TopRes_Rank']

## Xiangyu's version

In [ ]:
app = dash.Dash()

available_indicators = ['crime_rank', 'hospital_rank','early_education_rank', 'old_education_rank']
app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                id='First-care',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value = 'crime_rank'
            )
            ],
        style={'width': '48%', 'display': 'inline-block'}),
        
        html.Div([
            dcc.Dropdown(
                id='Second-care',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value = 'hospital_rank'
            )
            ],
        style={'width': '48%', 'display': 'inline-block'}),
        
        html.Button('Submit', id='button'),
        
        html.Div(dcc.Graph(id='indicator-graphic'))
    ]),
    #dcc.Graph(id='indicator-graphic')
])

@app.callback(
    dash.dependencies.Output('indicator-graphic', 'figure'),
    [dash.dependencies.Input('button', 'n_clicks')],
    [dash.dependencies.State('First-care', 'value'),
     dash.dependencies.State('Second-care', 'value')])
    
def update_graph(Button, First_care, Second_care):
    df = pd.DataFrame()
    df['City'] = human['City']
    df['First'] = human[First_care]
    df['Second'] = human[Second_care]
    df['Total'] = (df['First']+df['Second']).rank(ascending=1)
    df['reverse_rank'] = df['Total'].rank(ascending=0)
    df['text'] = df['City'] + '<br># Final Rank ' + (df['Total']).astype(str)
    df['longitude'] = human['longitude']
    df['latitude'] = human['latitude']
    
    limits = [(0,10),(10,20),(20,30),(30,40),(40,50)]
    colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
    cities = []


    for i in range(len(limits)):
        lim = limits[i]
        df_sub = df[lim[0]:lim[1]]
        city = dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = df_sub['longitude'],
            lat = df_sub['latitude'],
            text = df_sub['text'],
            marker = dict(
                size = df_sub['reverse_rank']*15,
                color = colors[i],
                line = dict(width=0.5, color='rgb(40,40,40)'),
                sizemode = 'area'
            ),
            name = '{0} - {1}'.format(lim[0],lim[1]) )
        cities.append(city)

        layout = dict(
            title = 'The human related ranking of US big cities',
            showlegend = True,
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showland = True,
                landcolor = 'rgb(217, 217, 217)',
                subunitwidth=1,
                countrywidth=1,
                subunitcolor="rgb(255, 255, 255)",
                countrycolor="rgb(255, 255, 255)"
            ),
        )
        fig = dict( data=cities, layout=layout )
        
    return {   
        plotly.offline.plot( fig, validate=False )     
    }

app.run_server()

## General version

In [24]:
app = dash.Dash()

# Get DataFrame value
available_indicators = available
app.layout = html.Div([
    
    html.Div([
        
        html.Center(html.H1("Which city would you like to live?", 
                            style={'color': 'blue', 'fontFace': 'Helvetica', 'fontSize': 25}
                           ),
                   ),
        
        html.Center(html.P("Powerd by City Fynders Group, University of Washington",
                           style={'color': 'pink', 'fontSize': 15}),
                   ),
        
        html.Br(),
        
        html.Center(html.Img(src="https://user-images.githubusercontent.com/32344254/32299959-b57f7bf6-bf14-11e7-8c03-90c0a7e29d0a.jpg")),
        
        
        # Different care classes represent different weight
        
        # First important factor
        html.Div([             
            
            html.P('1st'),
            
            dcc.Dropdown(
                id='First-care',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value = 'crime_rank'
            )
            ],
        style={'width': '20%', 'display': 'inline-block'}),
        
        
        # Second important factor
        html.Div([
            
            html.P('2nd'),
            
            dcc.Dropdown(
                id='Second-care',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value = 'hospital_rank'
            )
            ],
        style={'width': '20%', 'display': 'inline-block'}),
        
        # Third important factor
        html.Div([
            
            html.P('3rd'),
            
            dcc.Dropdown(
                id='Third-care',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value = 'hospital_rank'
            )
            ],
        style={'width': '20%', 'display': 'inline-block'}),
        
        # Fourth important factor
        html.Div([
            
            html.P('4th'),
            
            dcc.Dropdown(
                id='Fourth-care',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value = 'hospital_rank'
            )
            ],
        style={'width': '20%', 'display': 'inline-block'}),
        
        # Fifth important factor
        html.Div([
            
            html.P('5th'),
            
            dcc.Dropdown(
                id='Fifth-care',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value = 'hospital_rank'
            )
            ],
        style={'width': '20%', 'display': 'inline-block'}),
        
        html.Br(),
        
        html.Center(html.Button('Submit', id='button',
                                style={'width': 200, 'color': 'white', 'backgroundColor': 'green', 'height': 70})),
        
        html.Div(dcc.Graph(id='indicator-graphic'))
    ]),
    #dcc.Graph(id='indicator-graphic')
])

@app.callback(
    dash.dependencies.Output('indicator-graphic', 'figure'),
    [dash.dependencies.Input('button', 'n_clicks')],
    [dash.dependencies.State('First-care', 'value'),
     dash.dependencies.State('Second-care', 'value'),
     dash.dependencies.State('Third-care', 'value'),
     dash.dependencies.State('Fourth-care', 'value'),
     dash.dependencies.State('Fifth-care', 'value'),
    ]
)
    
def update_graph(Button, First_care, Second_care, Third_care, Fourth_care, Fifth_care):
    df = pd.DataFrame()
    df['City'] = alldata['City']
    df['First'] = alldata[First_care]
    df['Second'] = alldata[Second_care]
    df['Third'] = alldata[Third_care]
    df['Fourth'] = alldata[Fourth_care]
    df['Fifth'] = alldata[Fifth_care]
    df['Total'] = (df['First']*5+df['Second']*4+df['Third']*3+df['Fourth']*2+df['Fifth']*1).rank(ascending=1)
    df = df.sort_values('Total', ascending=1)
    df['reverse_rank'] = df['Total'].rank(ascending=0)
    df['text'] = df['City'] + '<br># Final Rank ' + (df['Total']).astype(str)
    df['longitude'] = alldata['Longitude']
    df['latitude'] = alldata['Latitude']
    
    limits = [(0,10),(10,20),(20,30),(30,40),(40,50)]
    colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
    cities = []


    for i in range(len(limits)):
        lim = limits[i]
        df_sub = df[lim[0]:lim[1]]
        city = dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = df_sub['longitude'],
            lat = df_sub['latitude'],
            text = df_sub['text'],
            marker = dict(
                size = df_sub['reverse_rank']*15,
                color = colors[i],
                line = dict(width=0.5, color='rgb(40,40,40)'),
                sizemode = 'area'
            ),
            name = '{0} - {1}'.format(lim[0],lim[1]) )
        cities.append(city)

        layout = dict(
            title = 'The human related ranking of US big cities',
            showlegend = True,
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showland = True,
                landcolor = 'rgb(217, 217, 217)',
                subunitwidth=1,
                countrywidth=1,
                subunitcolor="rgb(255, 255, 255)",
                countrycolor="rgb(255, 255, 255)"
            ),
        )
        fig = dict( data=cities, layout=layout )
        
    return {   
        plotly.offline.plot( fig, validate=False )     
    }

app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Nov/2017 16:24:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2017 16:24:20] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2017 16:24:20] "GET /_dash-dependencies HTTP/1.1" 200 -
[2017-11-22 16:24:20,564] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/han/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/han/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/han/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/han/anaconda3/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/Users/han/anaconda3/lib/python3.6/site-packages/flask/app.p

In [ ]:
dcc.Markdown('''

            Select the 4th important factor you considered... 
            
            '''